# For Instruction Induction dataset

In [ ]:
import json
all_datasets = 'active_to_passive antonyms auto_categorization auto_debugging cause_and_effect common_concept diff first_word_letter informal_to_formal larger_animal letters_list negation num_to_verbal odd_one_out object_counting orthography_starts_with periodic_elements rhymes second_word_letter sentence_similarity sentiment singular_to_plural sum synonyms taxonomy_animal translation_en-de translation_en-es translation_en-fr word_sorting word_unscrambling'
all_datasets = all_datasets.split()
new_taskes = []
for dataset_ in all_datasets:
	with open('./raw/induce/{}.json'.format(dataset_), 'r') as f:
		data = json.load(f)
		examples = data['examples']
	with open('./raw/execute/{}.json'.format(dataset_), 'r') as f:
		data_test = json.load(f)
	if len(examples) <= 100:
		print(dataset_)
		continue
	new_task_name = f"{dataset_}_v1"
	# divide the examples to train and valid where valid is 20% of the examples
	induce_size = int(0.8 * len(examples))
	valid_size = len(examples) - induce_size
	# Induce
	import copy
	data_train = copy.deepcopy(data)
	data_valid = copy.deepcopy(data)
	data_train['examples'] = {k: v for k, v in data['examples'].items() if int(k) <= induce_size}
	data_valid['examples'] = {str(int(k) - induce_size): v for k, v in data['examples'].items() if int(k) > induce_size}
	data_train['metadata']['num_examples'] = len(data_train['examples'])
	data_valid['metadata']['num_examples'] = len(data_valid['examples'])
	with open('./raw/induce/{}.json'.format(new_task_name), 'w') as f:
		json.dump(data_train, f, indent=4)
	with open('./raw/valid/{}.json'.format(new_task_name), 'w') as f:
		json.dump(data_valid, f, indent=4)
	with open('./raw/execute/{}.json'.format(new_task_name), 'w') as f:
		json.dump(data_test, f, indent=4)
	new_taskes += [new_task_name]

# Noisy dataset

In [4]:
tasks = ['sentence_similarity'] # Replace with all tasks
noisy_sizes = [10, 20, 30, 40, 50, 60, 70, 80, 90]

import os
import json
import random
import numpy as np

induce_size = 100
# noisy_size = 50
induce = True
induce_data_path = 'raw/induce/'
eval_data_path = 'raw/execute/'

for noisy_size in noisy_sizes:
	for task in tasks:
		with open('raw/induce/{}.json'.format(task), 'r') as f:
			data = json.load(f)

		# Induce
		np.random.seed(0)
		noisy_indices = np.random.randint(induce_size-noisy_size, induce_size, size=noisy_size)

		out = {}
		for i in range(induce_size):
			if i < induce_size - noisy_size:
				# Clean
				out[str(i+1)] = {
					'input': data['examples'][str(i+1)]['input'],
					'output': data['examples'][str(i+1)]['output'],
				}
			else:
				# Noisy
				out[str(i+1)] = {
					'input': data['examples'][str(i+1)]['input'],
					'output': data['examples'][str(noisy_indices[i-induce_size+noisy_size])]['output'],
				}

		dict = {"metadata": {
			"num_examples": len(out)
			},
				"examples": out
		}
		with open('raw/induce/{}_{}_noisy.json'.format(task, str(noisy_size)), 'x') as fp:
			json.dump(dict, fp, indent=4)
			
		# Valid
		valid_size = 100

		out = {}
		for i in range(valid_size):
			out[str(i+1)] = {
				'input': data['examples'][str(i+induce_size+1)]['input'],
				'output': data['examples'][str(i+induce_size+1)]['output'],
			}
			
		dict = {"metadata": {
			"num_examples": len(out)
			},
				"examples": out
		}
		with open('raw/valid/{}_{}_noisy.json'.format(task, str(noisy_size)), 'x') as fp:
			json.dump(dict, fp, indent=4)
			
		# Execute
		with open('raw/execute/{}.json'.format(task), 'r') as f:
			data = json.load(f)

		with open('raw/execute/{}_{}_noisy.json'.format(task, str(noisy_size)), 'x') as fp:
			json.dump(data, fp, indent=4)
		


# LR

In [ ]:
import os
import json
import random
import numpy as np

a=-4
b=6

def func(x1):
	ans = a*x1 + b
	print('Input: {}\nOutput: {}\n'.format(x1, ans))
	return ans

induce_size = 200
test_size = 100
task = 'linear_4'
induce = True
induce_data_path = 'raw/induce/'
eval_data_path = 'raw/execute/'

# Induce
np.random.seed(0)
inputs = np.random.randint(0, 500, induce_size)

out = {}
for i, j in enumerate(range(induce_size)):
	out[str(i+1)] = {
		'input': '{}'.format(inputs[i]),
		'output': '{}'.format(func(inputs[i])),
	}x
	
dict = {"metadata": {
	"num_examples": len(out)
	},
		"examples": out
}
with open('raw/induce/{}.json'.format(task), 'x') as fp:
	json.dump(dict, fp, indent=4)
	
# Execute
np.random.seed(1)
inputs = np.random.randint(0, 500, test_size)

out = {}
for i, j in enumerate(range(test_size)):
	out[str(i+1)] = {
		'input': '{}'.format(inputs[i]),
		'output': '{}'.format(func(inputs[i])),
	}
	
dict = {"metadata": {
	"num_examples": len(out)
	},
		"examples": out
}
with open('raw/execute/{}.json'.format(task), 'x') as fp:
	json.dump(dict, fp, indent=4)

In [ ]:
# NOISY

import os
import json
import random
import numpy as np

a=-4
b=6

c=5
d=-8

def func(x1):
	ans = a*x1 + b
	print('Input: {}\nOutput: {}\n'.format(x1, ans))
	return ans

def noisy_func(x1):
	ans = c*x1 + d
	print('Input: {}\nOutput: {}\n'.format(x1, ans))
	return ans

# noisy_sizes = [10, 20, 30, 40, 60, 70, 80, 90]
noisy_sizes = [0]

for noisy_size in noisy_sizes:
	induce_size = 100
	# noisy_size = 50
	valid_size = 100
	test_size = 100
	task = 'linear_4_{}_noisy'.format(noisy_size)
	induce = True
	induce_data_path = 'raw/induce/'
	eval_data_path = 'raw/execute/'

	# Induce
	np.random.seed(0)
	inputs = np.random.randint(0, 500, induce_size)

	out = {}
	for i, j in enumerate(range(induce_size)):
		out[str(i+1)] = {
			'input': '{}'.format(inputs[i]),
			'output': '{}'.format(func(inputs[i]) if i < induce_size - noisy_size else noisy_func(inputs[i])),
		}
		
	dict = {"metadata": {
		"num_examples": len(out)
		},
			"examples": out
	}
	with open('raw/induce/{}.json'.format(task), 'x') as fp:
		json.dump(dict, fp, indent=4)
		
	# Valid
	np.random.seed(1)
	inputs = np.random.randint(0, 500, valid_size)

	out = {}
	for i, j in enumerate(range(valid_size)):
		out[str(i+1)] = {
			'input': '{}'.format(inputs[i]),
			'output': '{}'.format(func(inputs[i])),
		}
		
	dict = {"metadata": {
		"num_examples": len(out)
		},
			"examples": out
	}
	with open('raw/valid/{}.json'.format(task), 'x') as fp:
		json.dump(dict, fp, indent=4)
		
	# Execute
	np.random.seed(2)
	inputs = np.random.randint(0, 500, test_size)

	out = {}
	for i, j in enumerate(range(test_size)):
		out[str(i+1)] = {
			'input': '{}'.format(inputs[i]),
			'output': '{}'.format(func(inputs[i])),
		}
		
	dict = {"metadata": {
		"num_examples": len(out)
		},
			"examples": out
	}
	with open('raw/execute/{}.json'.format(task), 'x') as fp:
		json.dump(dict, fp, indent=4)

# Pig-latin

In [2]:
def pig_latin_translator(original_sentence, end1='yay', end2='ay'):
    words = original_sentence.split()
    vowels = ["a", "e", "i", "o", "u", "A", "E", "I", "O", "U"]
    output = ""
    for word in words:
        # check if the word is capitalized
        capitalized = word[0].isupper()
        punctuation = ""
        if word[-1] in [".", ",", "!", "?"]:
            punctuation = word[-1]
            word = word[:-1]
        if word[0] in vowels:
            if word[-1] == end1[0]:
                translated_word = word + end1[1:]
            else:
                translated_word = word + f"{end1}"
        else:
            start = 0
            for i, letter in enumerate(word):
                if letter in vowels:
                    break
                else:
                    # if letter == "y":
                    #     break
                    # else:
                    start += 1
            translated_word = word[start:] + word[:start] + f"{end2}"
        if capitalized:
            translated_word = translated_word.capitalize()
        translated_word += punctuation
        output += translated_word + " "
    return output.strip()

print(pig_latin_translator("Every you fox, The jumps Over, the Lazy dog!"))

Everyay ouyay oxfay, Ethay umpsjay Overyay, ethay Azylay ogday!


In [54]:
import numpy as np
import json

task_name = 'pig_latin_variant2_encode'

with open('big-bench/pig_latin_encode.json'.format(task_name), 'r') as f:
	data = json.load(f)
	examples = data['examples']
for i in range(len(examples)):
	examples[i]['target'] = pig_latin_translator(examples[i]['input'], end1='ay', end2='ay')
	
# noisy_sizes = [10, 20, 30, 40, 60, 70, 80, 90]
noisy_sizes = [10, 30, 50, 70, 90]

for noisy_size in noisy_sizes:
	induce_size = 100 if noisy_size > 0 else 200
	valid_size = 100 if noisy_size > 0 else 0
	test_size = 100
	task = '{}_{}_noisy'.format(task_name, noisy_size) if noisy_size > 0 else task_name

	# Induce
	np.random.seed(0)
	noisy_indices = np.random.randint(induce_size-noisy_size, induce_size, size=noisy_size)

	# Induce
	out = {}
	for i, j in enumerate(range(induce_size)):
		out[str(i+1)] = {
			'input': '{}'.format(examples[i]['input']),
			'output': '{}'.format(examples[i]['target'] if i < induce_size - noisy_size else '{}'.format(examples[i]['input'])),
		}
		
	dict = {"metadata": {
		"num_examples": len(out),
		"description": data['description'],
		"task_prefix": data['task_prefix'],
		},
			"examples": out
	}
	with open('raw/induce/{}.json'.format(task), 'w') as fp:
		json.dump(dict, fp, indent=4)
		
	# Valid
	if valid_size > 0:
		out = {}
		for i, j in enumerate(range(valid_size)):
			out[str(i+1)] = {
				'input': '{}'.format(examples[induce_size+i]['input']),
				'output': '{}'.format(examples[induce_size+i]['target']),
			}
			
		dict = {"metadata": {
			"num_examples": len(out),
			"description": data['description'],
			"task_prefix": data['task_prefix'],
			},
				"examples": out
		}
		with open('raw/valid/{}.json'.format(task), 'w') as fp:
			json.dump(dict, fp, indent=4)
		
	# Execute
	out = {}
	for i, j in enumerate(range(test_size)):
		out[str(i+1)] = {
			'input': '{}'.format(examples[induce_size+valid_size+i]['input']),
			'output': '{}'.format(examples[induce_size+valid_size+i]['target']),
		}
		
	dict = {"metadata": {
		"num_examples": len(out),
		"description": data['description'],
		"task_prefix": data['task_prefix'],
		},
			"examples": out
	}
	with open('raw/execute/{}.json'.format(task), 'w') as fp:
		json.dump(dict, fp, indent=4)

# AG News Remap

In [ ]:
# ag news redirect1 with text label

from datasets import load_dataset
dataset = load_dataset("ag_news")

import numpy as np
import json

task_name='ag_news_textlabel_redirect1'

induce_size = 100 
valid_size = 100 
test_size = 100
label_map = {0:1, 1:2, 2:3, 3:0}
label_text = {0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}

noise_ratio = [10, 30, 50, 70, 90]
for noisy_size in noise_ratio:
	task = '{}_{}_noisy'.format(task_name, noisy_size)

	# shuffle the examples
	np.random.seed(0)
	perm = np.random.permutation(len(dataset['train']['text']))
	examples = []
	for i, j in enumerate(perm[:induce_size+valid_size+test_size]):
		examples.append({'input':dataset['train']['text'][j], 'target':label_text[label_map[dataset['train']['label'][j]]]})
	print("End")
	np.random.seed(0)
	noisy_indices = np.random.randint(induce_size-noisy_size, induce_size, size=noisy_size)
	# Induce
	out = {}
	for i, j in enumerate(range(induce_size)):
		out[str(i+1)] = {
			'input': '{}'.format(examples[i]['input']),
			'output': '{}'.format(examples[i]['target'] if i < induce_size - noisy_size else examples[noisy_indices[i-induce_size+noisy_size]]['target']),
		}
	dict = {"metadata": {
		"num_examples": len(out),
		"description": "AG news classification",
		"task_prefix": "",
		},
			"examples": out
	}
	with open('raw/induce/{}.json'.format(task), 'w') as fp:
		json.dump(dict, fp, indent=4)
		
	# Valid
	if valid_size > 0:
		out = {}
		for i, j in enumerate(range(valid_size)):
			out[str(i+1)] = {
				'input': '{}'.format(examples[induce_size+i]['input']),
				'output': '{}'.format(examples[induce_size+i]['target']),
			}
			
		dict = {"metadata": {
			"num_examples": len(out),
			"description": "AG news classification",
			"task_prefix": "",
			},
				"examples": out
		}
		with open('raw/valid/{}.json'.format(task), 'w') as fp:
			json.dump(dict, fp, indent=4)
		
	# Execute
	out = {}
	for i, j in enumerate(range(test_size)):
		out[str(i+1)] = {
			'input': '{}'.format(examples[induce_size+valid_size+i]['input']),
			'output': '{}'.format(examples[induce_size+valid_size+i]['target']),
		}
		
	dict = {"metadata": {
		"num_examples": len(out),
		"description": "AG news classification",
		"task_prefix": "",
		},
			"examples": out
	}
	with open('raw/execute/{}.json'.format(task), 'w') as fp:
		json.dump(dict, fp, indent=4)

# SST-5 Reverse

In [ ]:
# SST-5 remapping
from datasets import load_dataset

train = load_dataset("SetFit/sst5", split='train')

dataset = {}
dataset['train'] = train
# test = load_dataset("SetFit/sst5", split='test')

# {'text': "final verdict : you 've seen it all before .",
#  'label': 0,
#  'label_text': 'very negative'}

import numpy as np
import json

task_name='sst5_reverse'

induce_size = 100 
valid_size = 100 
test_size = 100
label_map = {0:4, 1:3, 2:2, 3:1, 4:0}
label_text = {0: 'very negative', 1: 'negative', 2: 'neutral', 3: 'positive', 4: 'very positive'}

noise_ratio = [0, 10, 30, 50, 70, 90]
# noise_ratio = [0]

for noisy_size in noise_ratio:
	task = '{}_{}_noisy'.format(task_name, noisy_size)

	# shuffle the examples
	np.random.seed(0)
	perm = np.random.permutation(len(dataset['train']['text']))
	examples = []
	for i, j in enumerate(perm[:induce_size+valid_size+test_size]):
		examples.append({'input':dataset['train']['text'][j], 'target':label_text[label_map[dataset['train']['label'][j]]]})
	print("End")
	np.random.seed(0)
	noisy_indices = np.random.randint(induce_size-noisy_size, induce_size, size=noisy_size)
	# Induce
	out = {}
	for i, j in enumerate(range(induce_size)):
		out[str(i+1)] = {
			'input': '{}'.format(examples[i]['input']),
			'output': '{}'.format(examples[i]['target'] if i < induce_size - noisy_size else examples[noisy_indices[i-induce_size+noisy_size]]['target']),
		}
	dict = {"metadata": {
		"num_examples": len(out),
		"description": "SST-5 classification reversed label",
		},
			"examples": out
	}
	with open('raw/induce/{}.json'.format(task), 'w') as fp:
		json.dump(dict, fp, indent=4)
		
	# Valid
	if valid_size > 0:
		out = {}
		for i, j in enumerate(range(valid_size)):
			out[str(i+1)] = {
				'input': '{}'.format(examples[induce_size+i]['input']),
				'output': '{}'.format(examples[induce_size+i]['target']),
			}
			
		dict = {"metadata": {
			"num_examples": len(out),
			"description": "SST-5 classification reversed label",
			"task_prefix": "",
			},
				"examples": out
		}
		with open('raw/valid/{}.json'.format(task), 'w') as fp:
			json.dump(dict, fp, indent=4)
		
	# Execute
	out = {}
	for i, j in enumerate(range(test_size)):
		out[str(i+1)] = {
			'input': '{}'.format(examples[induce_size+valid_size+i]['input']),
			'output': '{}'.format(examples[induce_size+valid_size+i]['target']),
		}
		
	dict = {"metadata": {
		"num_examples": len(out),
		"description": "SST-5 classification reversed label",
		"task_prefix": "",
		},
			"examples": out
	}
	with open('raw/execute/{}.json'.format(task), 'w') as fp:
		json.dump(dict, fp, indent=4)